In [72]:
#import libraries
import os
import csv
import requests
from pprint import pprint
from github import Github
from bs4 import BeautifulSoup

In [77]:
#write to csv file function
def write_page_to_file(page_tags):
    file = open('data.csv','a+',newline='')
    with file:
        write = csv.writer(file,delimiter=',')
        write.writerows(page_tags)

In [78]:
#find stack traces function
def find_stack_trace(url, tags):
    response_bug = requests.get(next_url)
    html_content_bug = response_bug.content
    dom_bug = BeautifulSoup(html_content_bug,'html.parser')
    temp = dom_bug.find('pre')
    if(temp != [] and temp !=None):
        if(temp.find('code')!=None):
            tags.append(str(temp.find('code')))

In [79]:
#search all closed issues.

#initialize important url's
base_url = "https://github.com/"
query_url = f"https://github.com/cgeo/cgeo/issues?page=1&q=is%3Aissue+is%3Aclosed"

#authorization in order to make more requests.
token = os.getenv("GITHUB_TOKEN")
headers = {'Authorization': f'token {token}'}

#make initial request
response = requests.get(query_url, headers = headers)


In [80]:
#loop through all pages
while True:
    response_code = response.status_code
    if response_code != 200:
        print("Error Occured")
        exit()
    else:
        html_content = response.content
        dom = BeautifulSoup(html_content,'html.parser')
        
        #find all issues in every page
        all = dom.findAll('div', class_='flex-auto min-width-0 p-2 pr-3 pr-md-2')
        
        #for every issue extract important informations
        page_tags = []
        
        for each in all:
            #store tags for every issue
            tags=[]
            is_Bug = False
            
            for counter,temp in enumerate(each.find_all('a')):
                tag = str(temp.string)
                if tag == "Bug":
                    is_Bug = True
                tags.append(tag)
            
            #if it is a Bug check for stack trace
            if(is_Bug == True):
                next_url = base_url + each.find('a')['href']
                find_stack_trace(next_url,tags)
            
            
            #store all issue's tags in a global struct 
            page_tags.append(tags)
            
        #write page issues to file
        write_page_to_file(page_tags)
    
    #visit next page if exists
    end = dom.findAll('a',class_='next_page')
    if end==[]:
        break
    next_url = base_url + end[1]['href']
    response = requests.get(next_url)